In [3]:
# Import utility libraries
from graphics import *
from bisect import bisect
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# for Jupyter
%matplotlib inline

# Reading and Preparing the Data

In [1]:
def data_from_file(name):
    """Read and parse the output of the relevant vcftools query
    
    We represent homozygous 0 as 0, heterozygous as 1 and homozygous 1 as 2
    The rare entries with a 2 in the vcf are represented arbitrarily as 0.
    """
    f = open("data/"+name,"r")
    s = f.read()
    f.close()
    lines = s.split("\n")
    data = []
    for l in lines:
        words = l.split()
        if len(words) == 2:
            pos = int(words[0])
            if (words[1] == '0/0') or '2' in words[1]:
                data.append((pos,0))
            elif words[1] == '0/1' or words[1] == '1/0':
                data.append((pos,1))
            elif words[1] == '1/1':
                data.append((pos,2))
    return sorted(data)

def read_files(files):
    """Read multiple files
    
    and put the results (and, for convenience the filenames)
    in a dictionary."""
    all_data = {}
    for f in files:
        all_data[f] = data_from_file(f)
    all_data["filenames"] = files
    return all_data

def add_diagnostics(data):
    """Add the list of diagnostic positions to the data."""
    diagnostic = set()
    for f in data["filenames"]:
        for p in data[f]:
            diagnostic.add(p[0])        
    data["diagnostics"] = sorted(diagnostic)

In [2]:
# Read the actual data
thefiles = ["IF_1303","IF_1306","IF_1308","IF_1322","IF_1325","IP_1330"]
all_data = read_files(thefiles)
add_diagnostics(all_data)

# Preparing the Chromosone Paintings
## Some basic colour manipulations

In [3]:
def rgb_color(col):
    """Convert a colour from Hex format into a list of three integers
    
    Opposite of the library function color_rgb."""
    return [int(col[1:3],16),int(col[3:5],16),
            int(col[5:7],16)]

def blend(colours, weights):
    """Blend colours together in proportion to weights
    
    Used to find the colour of each bucket in the painting."""
    totwt = sum(weights)
    if totwt == 0:
        return color_rgb(0,0,0)
    n = len(weights)
    cols = [rgb_color(c) for c in colours]
    b = [sum([cols[i][j]*weights[i] for i in range(n)])//totwt 
         for j in range(3)]
    return color_rgb(b[0],b[1],b[2])

def dim(color, ratio):
    """Adjust a colour's brightnes by a factor of ratio
    
    Was used in an earlier version of diagrams."""
    rgb = rgb_color(color)
    return color_rgb(int(rgb[0]*ratio),
                     int(rgb[1]*ratio),
                     int(rgb[2]*ratio))

# Red, yellow and blue -- representing homozygous 0, heterozygous and homozygous 1

basecols = [color_rgb(255,0,0), color_rgb(255,255,0),
            color_rgb(0,0,255)]

NameError: name 'color_rgb' is not defined

## Chromosone Painting

In [ ]:
def colours(scores, dodim):
    """Convert a triple of counts into a colour
    
    mixing the three base colours and (if dodim is set)
    adjusting brightness to reflect the number of 
    non-diagnostic sites in that block."""
    cols = [blend(basecols,s) for s in scores]
    if dodim:
        maxscore = max([sum(s) for s in scores])   
        return [dim(cols[i],sum(scores[i])/maxscore) 
                for i in range(len(scores))]
    return cols

### Version 1 -- including non-diagnostic sites

This version is no longer used in the final graphics-drawing functions below

In [ ]:
def score(data,buckets):
    """Count the number of diagnotic sites of each type in each bucket
    
    A bucket is just a range of positions on the chromosone."""
    maxind = data[-1][0]+1
    scores = [[0,0,0] for i in range(buckets)]
    for d in data:
        b = d[0]*buckets//maxind
        scores[b][d[1]] += 1
    return scores
  
def batch(data,buckets,dodim):
    """Proces a set of data into a sequence of colours."""
    return colours(score(data,buckets),dodim)
    

### Version 2 -- excluding non-diagnostic sites.

In [ ]:
def score2(data,buckets):
    """Count the diagnostic sites of each type within a bucket
    
    In this version the buckets divide up the diagnostic sites equally, 
    rather than dividing the whole chromosone equally."""
    dia = all_data["diagnostics"]
    block = len(dia)//buckets
    boundaries = [dia[block*i] for i in range(buckets)]
    boundaries.append(1000000000)
    scores = [[0,0,0] for i in range(buckets)]
    buck = 0;
    for d in data:
        while d[0] >= boundaries[buck+1]:
            buck += 1
        scores[buck][d[1]] += 1
    return scores
 
def batch2(data,buckets,dodim):
    """Process a set of data into a sequence of colours 
   
    using score2.""" 
    return colours(score2(data,buckets),dodim)

## Actual functions to draw the painted chromosones
These work with the score2 version above

In [ ]:
def drawDScale(win,top,left,w,dia):
    """Draw the even scale of diagnostic poitiont to go above the bars."""
    block = len(dia)//w
    Line(Point(left,top),Point(left+w, top)).draw(win)
    Text(Point(left+w//2 - 30, top -40),"Diagnostic Position Number").draw(win)
    for i in range(w//100+1):
        Line(Point(left+100*i,top),Point(left+100*i, top+20)).draw(win)
        Text(Point(left+100*i-5, top -20),str(block*i*100)).draw(win)
    for i in range(w//10+1):
        Line(Point(left+10*i,top),Point(left+10*i,top+10)).draw(win)
    win.update()


def drawAScale(win, bottom, left, w, dia):
    """Draw the uneven scale of  poition on the chromosone to go below the bars."""
    block = 50000    
    Line(Point(left,bottom),Point(left+w, bottom)).draw(win)
    Text(Point(left+w//2 - 30, bottom +40),"Absolute Position Number").draw(win)
    for i in range(dia[-2]//block):
        x = bisect(dia, i*block)*w//len(dia)
        if i % 10 == 0:
            Line(Point(left+x,bottom),
                 Point(left+x, bottom-20)).draw(win)
            Text(Point(left+x-5, bottom + 20),
                 str(block*i//1000)+"K").draw(win)
        else:
             Line(Point(left+x,bottom),
                 Point(left+x, bottom-10)).draw(win)
    win.update()             
                
def display(data,dodim,sep,margins,w,h):
    """Assemble the whole display
    
    This appears in a separate window. This graphics library doesn't appear 
    to be suypported by Jupyter for inline plotting.    """
    files = data["filenames"]
    nf = len(files)
    win = GraphWin(width=w+2*margins,
                   height=(h+sep)*nf - sep + 2*margins,
                   autoflush=False)
    for j in range(nf):        
        cols = batch2(data[files[j]], w, dodim)
        for i in range(w):
            l = Rectangle(Point(margins+i,margins+(h+sep)*j),
                          Point(margins+i,h+margins+(h+sep)*j))
            l.setOutline(cols[i])
            l.draw(win)
        win.update()
    drawAScale(win, 30 + margins + (h+sep)*nf - sep, margins, w, data["diagnostics"])
    drawDScale(win, margins - 30, margins, w, data["diagnostics"])
    return win


In [ ]:
#win.close()
win = display(all_data,False,20,100,2000,80)

# Calculate and Display Run Lengths
## Calculations

In [ ]:

def runs(data):
    """Divide data up into runs of the second component of each entry
    
    returns a list of (start position, length, value) tuples."""
    runs = []
    i = 0
    while i < len(data):
        runstart = i
        runval = data[i][1]
        while (i < len(data) and 
            data[i][1] == runval):
                i += 1
        runs.append((runstart, i - runstart, runval))
    return runs

def realrunlength(x,dia):
    """Calculate a run length in terms of actual base pairs
    
    rather than diagnostic sites."""
    return (x[0],x[1],dia[x[0]+x[1]] - dia[x[0]])

def hetrunlengths(runs,threshold,dia):
    """Extract the sequence of real lengths of heterogenous runs
    
    Ignores all intrusions of length <= threshold."""
    lens = []
    i = 0
    while i < len(runs): 
        if runs[i][2] == 1:
            l = runs[i][1]
            j = i+1
            skipped = 0
            while True:
                if j >= len(runs):
                    if l+skipped > threshold:
                        lens.append(realrunlength((runs[i][0],l+skipped),dia))
                    i = j
                    break
                if runs[j][2] != 1:
                    skipped += runs[j][1]
                    j += 1
                    if skipped > threshold:
                        if l > threshold:
                            lens.append(realrunlength((runs[i][0],l),dia))
                        i = j
                        break
                else: 
                    l += skipped + runs[j][1]
                    j += 1
                    skipped = 0
        else:
            i += 1
    return lens

llr = range(0,20) #range of log_2(length) that we consider 
               
                        
def rldist(rls):
    """Sort the run lengths into buckets
    
    Each bucket covers a power of 2. So the first bucket is < 8 then
    8..15, 16..32, etc."""
    pows = [2**i+0.5 for i in llr]
    return [bisect(pows,x[2]) for x in rls]

def freqs(data):
    """Count the freqency of each bucket."""
    f = [0 for i in llr]
    for d in data:
        f[d] += 1
    return f
        

## Display

In [ ]:

    
def plotfreqs(ax, f, title):
    """Plot a set of frwq"""
    pows = [2**x/1000.0 for x in llr]
    ax.bar(pows,f,width=[0.6*x for x in pows])
    ax.set_xscale('log')
    ax.set_xlabel("run length (kBases)")
    ax.tick_params(axis='y', labelleft = True)
    ax.set_ylabel("number of runs")
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
    ax.set_title(title)


def rlgraph(data,id, ax, thresh):
    """Plot a single runlength histogram on axes ax."""
    r = runs(data[id])
    h = hetrunlengths(r,thresh,data["diagnostics"])
    f = freqs(rldist(h))
    plotfreqs(ax,f,id)

def all_graphs(data, thresh):
    """Plot graphs for all six data sets."""
    fig,ax = plt.subplots(3,2, figsize=(15,15), sharey=True)
    fig.tight_layout(pad=5.0)
    rlgraph(data,'IF_1303',ax[0,0],thresh)
    rlgraph(data,'IF_1306',ax[0,1],thresh)
    rlgraph(data,'IF_1308',ax[1,0],thresh)
    rlgraph(data,'IF_1322',ax[1,1],thresh)
    rlgraph(data,'IF_1325',ax[2,0],thresh)
    rlgraph(data,'IP_1330',ax[2,1],thresh)
    return fig
    
def combined_graph(data, thresh):
    """plot a single graph combining the run length data from all data sets."""
    f = [freqs(rldist(hetrunlengths(runs(data[f]), thresh, data["diagnostics"]))) 
                          for f in data["filenames"]]
    sumfreqs = [sum([f[j][i] for j in range(len(f))]) 
                for i in range(len(f[1]))]
    fig, ax = plt.subplots(figsize = (15,15))
    plotfreqs(ax,sumfreqs, "Combined Run Lengths")
    return fig

In [ ]:
figall = all_graphs(all_data,2)
plt.savefig("separate.png")

In [ ]:
figcomb = combined_graph(all_data, 2)
plt.savefig("combined.png")